In [1]:
import pandas as pd
import torch
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import numpy as np
import karman
from karman.nn import *
import os
from torch.utils.data import Subset
from torch import nn
import argparse
from pyfiglet import Figlet
from termcolor import colored
from dataclasses import dataclass
from matplotlib import pyplot as plt
import imageio
from PIL import Image
import io
from IPython.display import Image
from astropy.time import Time
import astropy

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
time_and_position_based_features_to_calculate = [
    'all__day_of_year__[d]',
    'all__seconds_in_day__[s]',
    'all__sun_right_ascension__[rad]',
    'all__sun_declination__[rad]',
    'all__sidereal_time__[rad]',
    'tudelft_thermo__longitude__[deg]',
    'tudelft_thermo__latitude__[deg]'
    'tudelft_thermo__local_solar_time__[h]']

def create_datapoint(date, longitude, latitude, altitude):
    sample = {}
    date = pd.to_datetime(date)
    second_in_day = 3600*date.hour + 60*date.minute + date.second
    day_of_year = date.day_of_year
    t=Time(str(date),location=(f'{longitude}d', f'{latitude}d'))
    sunpos=astropy.coordinates.get_sun(t)
    sun_ra = sunpos.ra.rad
    sun_dec = sunpos.dec.rad
    side_real = t.sidereal_time('mean').rad
    lst = (second_in_day/3600.0 + longitude/15.)

    sample['all__seconds_in_day__[s]_sin'] = np.sin(2*np.pi*second_in_day/(24.0*60*60))
    sample['all__seconds_in_day__[s]_cos'] = np.cos(2*np.pi*second_in_day/(24.0*60*60))

    sample['all__day_of_year__[d]_sin'] = np.sin(2*np.pi*day_of_year/366.0)
    sample['all__day_of_year__[d]_cos'] = np.cos(2*np.pi*day_of_year/366.0)

    sample['all__sun_right_ascension__[rad]_sin'] = np.sin(sun_ra)
    sample['all__sun_right_ascension__[rad]_cos'] = np.cos(sun_ra)

    sample['all__sun_declination__[rad]_sin'] = np.sin(sun_dec)
    sample['all__sun_declination__[rad]_cos'] = np.cos(sun_dec)

    sample['all__sidereal_time__[rad]_sin'] = np.sin(side_real)
    sample['all__sidereal_time__[rad]_cos'] = np.cos(side_real)

    sample['tudelft_thermo__local_solar_time__[h]_sin'] = np.sin(2*np.pi*((lst + 12.0)/48))
    sample['tudelft_thermo__local_solar_time__[h]_cos'] = np.cos(2*np.pi*((lst + 12.0)/48))

    sample['tudelft_thermo__latitude__[deg]'] = float(latitude)

    sample['tudelft_thermo__longitude__[deg]_sin'] = np.sin(2*np.pi*((longitude + 180)/360.0))
    sample['tudelft_thermo__longitude__[deg]_cos'] = np.cos(2*np.pi*((longitude + 180)/360.0))

    sample['all__year__[y]'] = float(date.year)
    sample['tudelft_thermo__altitude__[m]'] = altitude
    return sample

In [5]:
model_path = '/home/jupyter/karman-project/data_directory/run_flare_lag_2880/best_model_NoFism2DailyFeedForward_2022-09-08 20:38:18.491403_fold_2_seed_0'
#Stick to this convention 'best_model' is the model of interest.
model_opt=torch.load(model_path)['opt']

In [8]:
model_dataset = karman.ThermosphericDensityDataset(
        directory='/home/jupyter/karman-project/data_directory',
        lag_minutes_omni=model_opt.lag_minutes_omni,
        lag_minutes_fism2_flare_stan_bands=model_opt.lag_fism2_minutes_flare_stan_bands,
        omni_resolution=model_opt.omni_resolution,
        fism2_flare_stan_bands_resolution=model_opt.fism2_flare_stan_bands_resolution,
        fism2_daily_stan_bands_resolution=model_opt.fism2_daily_stan_bands_resolution,
        features_to_exclude_thermo=model_opt.features_to_exclude_thermo.split(','),
        features_to_exclude_omni=model_opt.features_to_exclude_omni.split(','),
        features_to_exclude_fism2_flare_stan_bands=model_opt.features_to_exclude_fism2_flare_stan_bands.split(','),
        features_to_exclude_fism2_daily_stan_bands=model_opt.features_to_exclude_fism2_daily_stan_bands.split(','),
        create_cyclical_features=model_opt.cyclical_features,
    )
model_scaler = model_dataset.data_thermo['scaler']

Loading Omni.
Loading FISM2 Flare Stan bands.
Loading FISM2 Daily Stan bands.
Creating thermospheric density dataset
Creating cyclical features

Finished Creating dataset.


In [9]:
#have to do it like this to keep the correct scaler from the model run.
space_x_dataset = karman.ThermosphericDensityDataset(
        directory='/home/jupyter/karman-project/data_directory',
        lag_minutes_omni=model_opt.lag_minutes_omni,
        lag_minutes_fism2_flare_stan_bands=model_opt.lag_fism2_minutes_flare_stan_bands,
        omni_resolution=model_opt.omni_resolution,
        fism2_flare_stan_bands_resolution=model_opt.fism2_flare_stan_bands_resolution,
        fism2_daily_stan_bands_resolution=model_opt.fism2_daily_stan_bands_resolution,
        features_to_exclude_thermo=model_opt.features_to_exclude_thermo.split(','),
        features_to_exclude_omni=model_opt.features_to_exclude_omni.split(','),
        features_to_exclude_fism2_flare_stan_bands=model_opt.features_to_exclude_fism2_flare_stan_bands.split(','),
        features_to_exclude_fism2_daily_stan_bands=model_opt.features_to_exclude_fism2_daily_stan_bands.split(','),
        create_cyclical_features=model_opt.cyclical_features,
        max_date=pd.to_datetime('2022-06-01')
    )

Loading Omni.
Loading FISM2 Flare Stan bands.
Loading FISM2 Daily Stan bands.
Creating thermospheric density dataset
Creating cyclical features

Finished Creating dataset.


In [12]:
space_x_dataset.time_series_data['fism2_flare_stan_bands']['data'].head()

,fism2_flare_stan_bands__E0_05_0_4__[photons/cm**2/s],fism2_flare_stan_bands__E0_4_0_8__[photons/cm**2/s],fism2_flare_stan_bands__E0_8_1_8__[photons/cm**2/s],fism2_flare_stan_bands__E1_8_3_2__[photons/cm**2/s],fism2_flare_stan_bands__E3_2_7_0__[photons/cm**2/s],fism2_flare_stan_bands__E7_0_15_5__[photons/cm**2/s],fism2_flare_stan_bands__E15_5_22_4__[photons/cm**2/s],fism2_flare_stan_bands__E22_4_29_0__[photons/cm**2/s],fism2_flare_stan_bands__E29_0_32_0__[photons/cm**2/s],fism2_flare_stan_bands__E32_0_54_0__[photons/cm**2/s],...,fism2_flare_stan_bands__E79_8_91_3_low__[photons/cm**2/s],fism2_flare_stan_bands__E79_8_91_3_med__[photons/cm**2/s],fism2_flare_stan_bands__E79_8_91_3_high__[photons/cm**2/s],fism2_flare_stan_bands__E91_3_97_5_low__[photons/cm**2/s],fism2_flare_stan_bands__E91_3_97_5_med__[photons/cm**2/s],fism2_flare_stan_bands__E91_3_97_5_high__[photons/cm**2/s],fism2_flare_stan_bands__E97_5_98_7__[photons/cm**2/s],fism2_flare_stan_bands__E98_7_102_7__[photons/cm**2/s],fism2_flare_stan_bands__E102_7_105_0__[photons/cm**2/s],fism2_flare_stan_bands__E105_0_121_0__[photons/cm**2/s]
all__dates_datetime__,,,,,,,,,,,,,,,,,,,,,
2003-10-13 00:00:00,9537.197224,268857.862111,3.415363e+07,8.579697e+07,6.152347e+08,1.564234e+09,7.814994e+09,5.780004e+09,8.797233e+09,9.218649e+09,...,3.020788e+09,9.110898e+09,4.389259e+09,9.103338e+08,2.260406e+09,1.753422e+09,7.223733e+09,6.597644e+09,7.610754e+09,3.164182e+10
2003-10-13 00:30:00,9447.646047,271250.815862,3.437118e+07,8.623590e+07,6.178426e+08,1.569459e+09,7.839831e+09,5.804002e+09,8.814387e+09,9.246465e+09,...,3.026229e+09,9.126623e+09,4.396311e+09,9.115037e+08,2.263505e+09,1.756247e+09,7.229472e+09,6.607150e+09,7.620247e+09,3.168943e+10
2003-10-13 01:00:00,9196.903000,269397.509711,3.433389e+07,8.616931e+07,6.175403e+08,1.569036e+09,7.837834e+09,5.802141e+09,8.813119e+09,9.244285e+09,...,3.025817e+09,9.125481e+09,4.395785e+09,9.114230e+08,2.263291e+09,1.756057e+09,7.229077e+09,6.606495e+09,7.619585e+09,3.168554e+10
2003-10-13 01:30:00,9376.005269,269981.550397,3.430554e+07,8.610688e+07,6.172474e+08,1.568620e+09,7.835873e+09,5.800322e+09,8.811895e+09,9.242157e+09,...,3.025416e+09,9.124375e+09,4.395276e+09,9.113457e+08,2.263085e+09,1.755875e+09,7.228700e+09,6.605868e+09,7.618949e+09,3.168182e+10
2003-10-13 02:00:00,10011.818700,273154.274082,3.428662e+07,8.604873e+07,6.169634e+08,1.568210e+09,7.833942e+09,5.798538e+09,8.810710e+09,9.240074e+09,...,3.025024e+09,9.123301e+09,4.394779e+09,9.112712e+08,2.262887e+09,1.755701e+09,7.228336e+09,6.605264e+09,7.618336e+09,3.167853e+10


In [72]:
latitude = 0
longitude = 0
altitude = 200_000
sample_data = create_datapoint(pd.to_datetime('2022-02-01'), longitude, latitude, altitude)

date = pd.to_datetime('2022-02-01')
location_nearest_date = (space_x_dataset.data_thermo['data']['all__dates_datetime__']-date).abs().argsort()[:1]
data = space_x_dataset.data_thermo['data'].iloc[location_nearest_date, :].drop(columns=model_opt.features_to_exclude_thermo.split(',')).copy()
for key, value in sample_data.items():
    data.at[location_nearest_date, key] = value

fism2_date_lag = date - pd.Timedelta(minutes=model_opt.lag_fism2_minutes_flare_stan_bands)
fism2_flare_data = space_x_dataset.time_series_data['fism2_flare_stan_bands']['data'].loc[fism2_date_lag:date, :].copy()

omni_date_lag = date - pd.Timedelta(minutes=model_opt.lag_minutes_omni)
omni_data = space_x_dataset.time_series_data['omni']['data'].loc[omni_date_lag:date, :].copy()

print(len(model_dataset.features_to_exclude_thermo))
print(len(model_dataset.cyclical_features))
print(len(data.columns))
thermo_features = model_dataset.data_thermo['scaler'].transform(data)
fism2_flare_features = model_dataset.time_series_data['fism2_flare_stan_bands']['scaler'].transform(fism2_flare_data)
omni_features = model_dataset.time_series_data['omni']['scaler'].transform(omni_data)

batch = {}
batch['omni'] = torch.FloatTensor(omni_features).unsqueeze(0)
batch['fism2_flare_stan_bands'] = torch.FloatTensor(fism2_flare_features).unsqueeze(0)
batch['instantaneous_features'] = torch.FloatTensor(thermo_features.flatten()).unsqueeze(0)

20
7
33


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but QuantileTransformer was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but QuantileTransformer was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [75]:
if model_opt.model == 'NoFism2DailyFeedForward':
    model = NoFism2DailyFeedForward(
        dropout=model_opt.dropout,
        hidden_size=model_opt.hidden_size,
        out_features=model_opt.out_features
        ).to(dtype=torch.float32)

    state_dict = torch.load(os.path.join(model_path))['state_dict']
    #Sanitize state_dict key names
    for key in list(state_dict.keys()):
        if key.startswith('module'):
        # Model was saved as dataparallel model
            # Remove 'module.' from start of key
            state_dict[key[7:]] = state_dict.pop(key)
        else:
            continue
    with torch.no_grad():
        model.forward(batch)
    model.load_state_dict(state_dict)
    print('Loaded state dict')

with torch.no_grad():
    output = model(batch)

model_dataset.unscale_density(output)

Loaded state dict


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor([[1.6872e-10]])

In [79]:
from tqdm import tqdm
densities = []

for date in tqdm(list(pd.date_range(start='2022-01-30', end='2022-02-05', freq='1440T'))):
    latitude = 0
    longitude = 0
    altitude = 200_000
    sample_data = create_datapoint(date, longitude, latitude, altitude)
    location_nearest_date = (space_x_dataset.data_thermo['data']['all__dates_datetime__']-date).abs().argsort()[:1]
    data = space_x_dataset.data_thermo['data'].iloc[location_nearest_date, :].drop(columns=model_opt.features_to_exclude_thermo.split(',')).copy()
    for key, value in sample_data.items():
        data.at[location_nearest_date, key] = value

    fism2_date_lag = date - pd.Timedelta(minutes=model_opt.lag_fism2_minutes_flare_stan_bands)
    fism2_flare_data = space_x_dataset.time_series_data['fism2_flare_stan_bands']['data'].loc[fism2_date_lag:date, :].copy()

    omni_date_lag = date - pd.Timedelta(minutes=model_opt.lag_minutes_omni)
    omni_data = space_x_dataset.time_series_data['omni']['data'].loc[omni_date_lag:date, :].copy()

    thermo_features = model_dataset.data_thermo['scaler'].transform(data)
    fism2_flare_features = model_dataset.time_series_data['fism2_flare_stan_bands']['scaler'].transform(fism2_flare_data)
    omni_features = model_dataset.time_series_data['omni']['scaler'].transform(omni_data)

    batch = {}
    batch['omni'] = torch.FloatTensor(omni_features).unsqueeze(0)
    batch['fism2_flare_stan_bands'] = torch.FloatTensor(fism2_flare_features).unsqueeze(0)
    batch['instantaneous_features'] = torch.FloatTensor(thermo_features.flatten()).unsqueeze(0)
    with torch.no_grad():
        output = model(batch)
    densities.append(model_dataset.unscale_density(output))

  0%|          | 0/25 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but QuantileTransformer was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but QuantileTransformer was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
  4%|▍         | 1/25 [00:02<01:11,  2.96s/it]/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/opt/conda/lib/python3.7/s

In [ ]:
from matplotlib import pyplot as plt
date_start = '2022-01-01'
date_end = '2022-02-28'

for column in dataset.time_series_data['omni']['data'].columns:
    dataset.time_series_data['omni']['data'].loc[date_start:date_end].plot(y=[column])
    plt.xlabel('')
    plt.show()

dataset.time_series_data['omni']['data']['mag_bsn_pos'] = dataset.time_series_data['omni']['data']['omniweb__bsnz_gse__[Re]']**2 + \
    dataset.time_series_data['omni']['data']['omniweb__bsnx_gse__[Re]']**2 +\
    dataset.time_series_data['omni']['data']['omniweb__bsny_gse__[Re]']**2

dataset.time_series_data['omni']['data'].loc[date_start:date_end].plot(y=['mag_bsn_pos'])
plt.xlabel('')
plt.show()

for column in dataset.time_series_data['fism2_flare_stan_bands']['data'].columns:
    dataset.time_series_data['fism2_flare_stan_bands']['data'].loc[date_start:date_end].plot(y=[column])
    plt.xlabel('')
    plt.show()